In [1]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [2]:
dataset_path = "/workspace/data/data-bin/dataset_200k.bpe.diff-msg/"
dest_path = "/workspace/models/dataset_200k.bpe.transformer.diff-msg/"
tb_log_path = "/workspace/log/dataset_200k.bpe.transformer.diff-msg/"

## Train a baseline transformer_iwslt_de_en model

In [3]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    $dataset_path \
    --save-dir $dest_path \
    --keep-last-epochs 10 \
    --source-lang diff --target-lang msg \
    --arch transformer_iwslt_de_en \
    --activation-fn relu --attention-dropout 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 1e-5 --warmup-init-lr 5e-6 --lr-scheduler inverse_sqrt --warmup-updates 4000 --min-lr 5e-07 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 16384 \
    --tensorboard-logdir $tb_log_path \
    --fp16

/bin/sh: 1: fairseq-train: not found


In [ ]:
import os
model_path = os.path.join(dest_path, "checkpoint_last.pt")
test_eval_path = os.path.join(dest_path, "test-eval.out")

## Evaluate baseline translation result on test subset

In [ ]:
!fairseq-generate \
    $dataset_path \
    --path $model_path \
    --beam 15 --lenpen 0.5 --remove-bpe --bpe sentencepiece \
    --fp16 \
    --batch-size 64 \
    --gen-subset test | tee $test_eval_path